In [ ]:
# Cell 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import json

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Cell 2 - Load models and data
logistic_model = joblib.load('../models/logistic_regression.joblib')
rf_model = joblib.load('../models/random_forest.joblib')
xgb_model = joblib.load('../models/xgboost.joblib')

# Load test data (you'll need to save this in training script)
# For now, reload and split
df = pd.read_csv('../data/telco_churn_features.csv')
from sklearn.model_selection import train_test_split

X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Cell 3 - Compare ROC curves
fig, ax = plt.subplots(figsize=(10, 8))

models = {
    'Logistic Regression': logistic_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model
}

for name, model in models.items():
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    ax.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Cell 4 - Feature importance (Random Forest)
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False).head(15)

plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance, y='feature', x='importance')
plt.title('Top 15 Most Important Features (Random Forest)', fontsize=14, fontweight='bold')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

# Cell 5 - Confusion matrices comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, model) in enumerate(models.items()):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(f'{name}\nConfusion Matrix')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

# Cell 6 - Model metrics comparison
metrics_data = []
for name in ['logistic_regression', 'random_forest', 'xgboost']:
    with open(f'../models/{name}_metrics.json', 'r') as f:
        metrics = json.load(f)
        metrics['model'] = name
        metrics_data.append(metrics)

metrics_df = pd.DataFrame(metrics_data)
metrics_df = metrics_df[['model', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]
metrics_df = metrics_df.round(4)

print("Model Performance Comparison:")
print("=" * 80)
print(metrics_df.to_string(index=False))